In [1]:
import stgen
import torch as th
import torchsummary
from stgen import utils
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

model_name='t5-small'
t5_tokenizer=stgen.tokenizer(model_name)
text=['isthis true' ,'what is happening']
x,y=t5_tokenizer.get_tokens_masks(text)
device= th.device('cuda')
batch_size=len(text)


c:\users\imagry\desktop\linux\converter\lib\site-packages\transformers\models\t5\tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
model=stgen.stgen(model_name,512,batch_size)
hidden=model.initHidden().to(device)
model=model.to(device)
output,hidden = model(x,y, hidden)

## Data

In [17]:
import pandas as pd
from stgen import utils
from torch.utils.data import DataLoader, TensorDataset
batch_size=64

In [3]:
df=pd.read_csv('./data/startups.csv')#[0:1000]


In [5]:
train_set,test_set=utils.tts(df,0.15)
train_set = list(train_set.to_records(index=False))
test_set = list(test_set.to_records(index=False))
train_set=[S for S in train_set if (str(S[0]) !='nan' and str(S[2]) !='nan')]
test_set=[S for S in test_set if (str(S[0]) !='nan' and str(S[2]) !='nan')]

In [6]:
Train_Y      = np.array([utils.lineToTensor(utils.pad_word(sample[0])).squeeze().cpu().numpy() for sample in train_set])
Train_Tokens = np.array([t5_tokenizer.get_tokens_masks([sample[2]])[0].squeeze().cpu().numpy() for sample in train_set ])
Train_Masks  = np.array([t5_tokenizer.get_tokens_masks([sample[2]])[1].squeeze().cpu().numpy() for sample in train_set ])

Test_Y       = np.array([utils.lineToTensor(utils.pad_word(sample[0])).squeeze().cpu().numpy() for sample in test_set  ])
Test_Tokens  = np.array([t5_tokenizer.get_tokens_masks([sample[2]])[0].squeeze().cpu().numpy() for sample in test_set  ])
Test_Masks   = np.array([t5_tokenizer.get_tokens_masks([sample[2]])[1].squeeze().cpu().numpy() for sample in test_set  ])


In [18]:
print(Train_Y.shape)
print(Train_Tokens.shape)
print(Train_Masks.shape)
print(Test_Y.shape)
print(Test_Tokens.shape)
print(Test_Masks.shape)

(31215, 20, 59)
(31215, 256)
(31215, 256)
(5515, 20, 59)
(5515, 256)
(5515, 256)


In [19]:
train_ds = TensorDataset(th.from_numpy(Train_Tokens),th.from_numpy(Train_Masks), th.from_numpy(Train_Y))
test_ds  = TensorDataset(th.from_numpy(Test_Tokens),th.from_numpy(Test_Masks), th.from_numpy(Test_Y))

In [20]:
train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=False)
test_dl  = DataLoader(test_ds, shuffle=True, batch_size=batch_size, drop_last=False)


# Training  :) 

In [14]:
from torch import nn
from torch.autograd import Variable

In [ ]:
model=stgen.stgen(model_name,512,batch_size)
#output,hidden = model(x,y, hidden)

model=model.to(device)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = th.optim.Adam(model.parameters(), lr = 1e-3)
epochs = 6
losses = []
th.autograd.set_detect_anomaly(True)


for e in range(epochs):

    for batch_idx, batch in enumerate(train_dl):

        tokens = batch[0].to(device)
        masks  = batch[1].to(device)
        targets= batch[2].to(device)
        
        

        optimizer.zero_grad()
        hidden=model.initHidden().to(device)
        
        with th.set_grad_enabled(True):
            output=[]
            
            for i in range(20):
                out, hidden = model(tokens,masks,hidden)
                output.append(out)
            output=th.cat((output),1)
            output=th.reshape(output, (batch_size,20, -1))
            #print(output.shape,targets.shape)
            loss = criterion(output, targets)
            
            if batch_idx%5==0:
                print(loss)
            
            
            loss.backward(retain_graph=True)
            optimizer.step()
            
           
            
    losses.append(loss.item())

Some weights of the model checkpoint at t5-small were not used when initializing T5EncoderModel: ['decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.2.layer.1.layer_norm.weight', 'decoder.final_layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.0.SelfAttention.v.weight', 'decoder.block.4.layer.2.DenseReluDense.wi.weight', 'decoder.block.2.layer.1.EncDecAttention.v.weight', 'decoder.block.4.layer.2.layer_norm.weight', 'decoder.block.1.layer.2.layer_norm.weight', 'decoder.block.3.layer.2.DenseReluDense.wi.weight', 'decoder.block.2.layer.1.EncDecAttention.o.weight', 'decoder.block.2.layer.0.SelfAttention.v.weight', 'decoder.block.1.layer.1.EncDecAttention.v.weight', 'decoder.block.5.layer.1.EncDecAttention.k.weight', 'decoder.block.1.layer.1.layer_norm.weight', 'decoder.block.5.layer.0.SelfAttention.k.weight', 'decoder.block.4.layer.1.EncDecAttention.q.weight', 'decoder.block.5.layer.0.SelfAttention.o.weight', 'decoder.block.0.laye

In [ ]:
th.empty((0))